In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import os

# Configurações
model_name = "BAAI/bge-m3"
output_folder = 'vectorstore'
existing_db_path = 'vectorstore/db_faiss_all_pdfs'

# Função para carregar e processar PDFs
def load_and_process_pdfs(pdf_folder):
    documents = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            file_path = os.path.join(pdf_folder, filename)
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    split_docs = text_splitter.split_documents(documents)
    return split_docs

# Carregar ou criar o banco de vetores
def get_vectorstore(pdf_folder):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    if os.path.exists(existing_db_path):
        return FAISS.load_local(existing_db_path, embeddings, allow_dangerous_deserialization=True)
    else:
        docs = load_and_process_pdfs(pdf_folder)
        vectorstore = FAISS.from_documents(docs, embeddings)
        vectorstore.save_local(existing_db_path)
        return vectorstore


vectorstore = get_vectorstore('pdf_ppg')
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'fetch_k': 6})

c:\Users\nielsen.castelo\AppData\Local\anaconda3\envs\inatel\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
def model_ollama(model="llama3.2", temperature=0.1):
    return ChatOllama(model=model, temperature=temperature)

llm = model_ollama()

In [3]:
context_q_system_prompt = """
Dado o histórico de chat e a pergunta de acompanhamento, que pode fazer referência ao contexto no histórico do chat, 
formule uma pergunta independente que possa ser entendida sem o histórico do chat. 
NÃO responda à pergunta, apenas reformule-a se necessário e, caso contrário, retorne-a como está.
"""

qa_prompt_template = """
Você é um assistente virtual especializado em analisar e responder perguntas sobre artigos científicos, base de dados utilizados, algoritmos, 
metodologia, avaliações, pontos fortes e fracos dos estudos sobre os artigos.
Use os seguintes trechos de contexto recuperado para responder à pergunta.
Se você não souber a resposta, diga honestamente que não sabe. Mantenha a resposta concisa e focada.
Se for solicitado a listar algoritmos, base de dados, métodos ou informações específicas dos artigos, faça-o de forma estruturada e clara.
Responda em português.

Pergunta: {input}

Contexto: {context}

Resposta:
"""

context_q_prompt = ChatPromptTemplate.from_messages([
    ("system", context_q_system_prompt),
    ("human", "Pergunta: {input}")
])

history_aware_retriever = create_history_aware_retriever(llm=llm, retriever=retriever, prompt=context_q_prompt)

qa_prompt = PromptTemplate.from_template(qa_prompt_template)
qa_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

# Função para fazer perguntas
def ask_question(question, chat_history=''):
    result = rag_chain.invoke({"input": question, "chat_history": chat_history})
    return result['answer']

In [4]:
user_query = 'Liste todos os algoritmos utilizados nos artigos sua base de conhecimento, mostre qual artigo da base de conhecimento foi utilizado por cada algoritmo'
response = ask_question(user_query)
print(response)

Olá! Estou aqui para ajudar com suas perguntas sobre artigos científicos.

Você solicitou uma lista de algoritmos utilizados nos artigos da base de conhecimento, juntamente com a referência do artigo correspondente. Aqui está a lista:

* **Logística Linear (LR)**: Utilizado em [21] "Assessment of vasoactive agents and vascular aging by the second derivative of photoplethysmogram waveform" e [3] "Assessing Elevated Blood Glucose Levels Through Blood Glucose Evaluation and Monitoring Using Machine Learning and Wearable Photoplethysmography Sensors: Algorithm Development nd Validation".
* **Rede Neuronal Convolucional (CNN)**: Utilizado em [9] "Detection of type-2 diabetes using characteristics of toe photoplethysmogram by applying support vector machine" e [33] "MACHINE LEARNING -BASED DIABETES DETECTION USING PHOTOPLETHYSMOGRAPHY SIGNAL FEATURES".
* **Rede Neuronal Deep (DNN)**: Utilizado em [8] "Detection of type-2 diabetes using characteristics of toe photoplethysmogram by applying su

In [5]:
user_query = 'Baseado na sua base de conhecimento Construa uma metodologia baseado nos melhores resultados'
response = ask_question(user_query)
print(response)

Com base nos artigos mencionados, aqui está uma metodologia proposta para o estudo de predição de diabetes tipo II:

**Passo 1: Preprocessamento dos dados**

* Centro os dados em zero média e escale-os para variância unitária.
* Aplicar SMOTE para corrigir a desigualdade de classes.

**Passo 2: Seleção de características**

* Utilizar uma abordagem de seleção de características baseada em feature subset selection, onde:
 + Crie múltiplos conjuntos de características que contenham as características mais importantes para a predição de diabetes.
 + Treine um modelo de classificação (XGBoost) para cada conjunto de características e avalie o desempenho do modelo utilizando métricas de precisão e F1-score.

**Passo 3: Avaliação do desempenho**

* Utilizar uma abordagem de avaliação baseada em cross-validation, onde:
 + Divida os dados em cinco conjuntos de treinamento e teste.
 + Treine o modelo utilizando cada conjunto de treinamento e avalie o desempenho do modelo utilizando métricas de p

In [6]:
user_query = 'Na sua base de conhecimento. Qual artigo utiilzou nos estudos o Shannon  Energy? veja qual o motivo'
response = ask_question(user_query)
print(response)

O artigo que utilizou o Shannon Energy é "Assessment of vasoactive agents and vascular aging by the second derivative of photoplethysmogram waveform" de Thambarawita, Jhah, Hamme, Johannesen e Halvorsen (2020).

A razão pelo qual o Shannon Energy foi utilizado nesse estudo é para quantificar a distribuição da energia do sinal de frequência em diferentes componentes. Isso ajuda a capturar a energia total do sinal e entender como ela está distribuída, o que pode ser útil para avaliar a saúde vascular.

Aqui estão os detalhes sobre o Shannon Energy:

*   Definição: A energia de Shannon é uma medida da energia total de um sinal de frequência.
*   Cálculo: É calculada dividindo a energia total do sinal por sua amplitude máxima.
*   Uso: Foi utilizado nesse estudo para avaliar a saúde vascular e identificar agentes vasoativos eficazes.

Essa medida é importante porque pode ajudar a entender melhor como a energia do sinal de frequência está relacionada à saúde vascular, o que pode ser útil pa

In [7]:
user_query = 'Na sua base de conhecimento. Qual são os atribultos utilizados para treinar os modelos usando sinais PPG?'
response = ask_question(user_query)
print(response)

Com base nos trechos de contexto fornecidos, os atributos utilizados para treinar os modelos usando sinais PPG incluem:

* Coeficientes de variação (variance)
* Coeficientes de skewness (skewness)
* Coeficientes de kurtosidade (kurtosis)
* Desvios padrão (standard deviation)
* Entropia da informação (information entropy)
* Idade
* Altura
* Peso
* Taxa cardíaca (heart rate)
* Fluxo sanguíneo (blood flow rate)
* Hemoglobina (hemoglobin)
* Oximetria pulmonar (pulse oximetry)

Além disso, alguns estudos também utilizam características derivadas do sinal PPG, como:

* Derivadas do sinal PPG
* Transformada de Fourier (FFT)

Os modelos utilizados para treinar os dados incluem:

* Redes neurais artificiais (BPNN)
* Redes neurais profundas (DNN)
* Convolução neural (CNN)
* Classificação por suporte vectorial máxima (GSVM)
* XGBoost
* Regressão logística

Os algoritmos utilizados para classificar os dados incluem:

* GSVM
* XGBoost
* Regressão logística

As bases de dados utilizadas incluem:

* 